In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import h5py
import tensorflow as tf
import os

from src.utils import to_rgb
from src.satellite_images.storage import SentinelDataset, SentinelDatasetIterator
from src.mask.mask_dataset import MaskDataset, MaskDatasetIterator
from src.mask.utils import apply_mask_to_image_series


data_path = '../../../kornmo-data-files/raw-data/crop-classification-data/'

In [2]:
print("Reading data")
data = gpd.read_file(f"{data_path}/training_data.gpkg")
data['orgnr'] = data['orgnr'].astype(int)
data['year'] = data['year'].astype(int)
print("Reading masks")
masks = MaskDataset(f"{data_path}/train_data_masks.h5")
print("Reading satellite images")
satellite_imgs = SentinelDataset('E:/MasterThesisData/Satellite_Images/satellite_images_train.h5')



Reading data
Reading masks
Reading satellite images


In [5]:
# print(data)
# print(data.loc[data['orgnr'] == 811935662].loc[data['year'] == 2019]['planted'].iloc[0])
# print(labels.index('rug'))
# print(data['planted'].count_values())
labels = list(set(data['planted']))
print(labels)
arr = [0 for i in range(0, len(labels))]
arr[2] = 1
print(arr)
def add_labels(orgnr, year, data_arg):
    # print(type(orgnr))
    # print(type(year))
    label = data.loc[data['orgnr'] == int(orgnr)].loc[data['year'] == int(year)]['planted'].iloc[0]
    index = labels.index(label)
    arr = [0 for i in range(0, len(labels))]
    arr[index] = 1
    return {'class': arr}

['rughvete', 'bygg', 'oljefro', 'rug', 'erter', 'havre', 'hvete']
[0, 0, 1, 0, 0, 0, 0]


In [6]:
train, val = satellite_imgs.to_iterator().split(rand_seed='corn')
train = train.with_data(add_labels)
val = val.with_data(add_labels)
masks_it = masks.get_iterator()
mask_dict = {}

for orgnr, year, mask in masks_it:
    mask_dict[f'{orgnr}/{year}'] = mask


train = train.filter(lambda orgnr, year, _,__: f"{orgnr}/{year}" in mask_dict)
val = val.filter(lambda orgnr, year, _,__: f"{orgnr}/{year}" in mask_dict)

print(f"train samples: {len(train)}")
print(f"val samples: {len(val)}")


train samples: 7844
val samples: 1961


In [11]:
#import random

def apply_mask(orgnr, year, imgs):
    mask = mask_dict[f'{orgnr}/{year}']
    return apply_mask_to_image_series(mask, imgs)

def apply_output(orgnr, year, img_source, data):
    label = data['class']
    #num = random.randint(0, 29)
    return {"cnn_input": img_source[0:30]}, label

In [12]:
from tensorflow.python.data.experimental import assert_cardinality

train_dataset = tf.data.Dataset.from_generator(
    train.transform(apply_mask).apply(apply_output).shuffled(),
    output_types=({'cnn_input': tf.dtypes.float64}, tf.dtypes.int64)
).apply(assert_cardinality(len(train)))

val_dataset = tf.data.Dataset.from_generator(
    val.transform(apply_mask).apply(apply_output).shuffled(),
    output_types=({'cnn_input': tf.dtypes.float64}, tf.dtypes.int64)
).apply(assert_cardinality(len(val)))

print(f"training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")

training samples: 7844
Validation samples: 1961


In [13]:
from keras import layers, models


def cnn(input_dim, output_dim):
    input_layer = layers.Input(shape=input_dim)
    y = layers.Conv2D(16, (3, 3), activation=tf.nn.relu, padding='same')(input_layer)
    y = layers.MaxPool2D((2, 2))(y)
    y = layers.Conv2D(32, (3, 3), activation=tf.nn.relu, padding='same')(y)
    y = layers.MaxPool2D((2, 2))(y)
    y = layers.Conv2D(64, (3, 3), activation=tf.nn.relu, padding='same')(y)
    y = layers.MaxPool2D((2, 2))(y)
    y = layers.Flatten()(y)
    y = layers.Dense(output_dim, activation=tf.nn.relu)(y)
    return models.Model(inputs=[input_layer], outputs=[y], name="SingleImageCNN")

In [14]:
from keras.models import load_model
from tensorflow import optimizers
from src.kornmo.pyimagesearch.callbacks.epochcheckpoint import EpochCheckpoint
from src.kornmo.pyimagesearch.callbacks.trainingmonitor import TrainingMonitor

plotPath = os.path.sep.join(["training", "hybrid_more_features.png"])
jsonPath = os.path.sep.join(["training", "hybrid_more_features.json"])
callbacks = [
	EpochCheckpoint('./training', every=1, startAt=0),
	TrainingMonitor(plotPath, jsonPath=jsonPath, startAt=0)]

restart = True
if restart:
    cnn_net = cnn((100, 100, 12), 64)
    input_cnn = layers.Input(shape=(30, 100, 100, 12), name="cnn_input")

    cnn = layers.TimeDistributed(cnn_net)(input_cnn)
    cnn = layers.GRU(128, return_sequences=False)(cnn)
    cnn = layers.Flatten()(cnn)
    cnn = layers.Dense(128)(cnn)
    cnn = layers.Dense(7, activation='softmax')(cnn)

    cnn = models.Model(inputs=input_cnn, outputs=cnn, name="CNN")

    cnn.compile(
        optimizer=optimizers.Adam(),
        loss=tf.keras.losses.CategoricalCrossentropy(),
        metrics=['categorical_accuracy']
    )
    # callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)


    cnn_history = cnn.fit(
            train_dataset.take(10000).batch(32).prefetch(2),
            validation_data=val_dataset.batch(32).prefetch(2),
            epochs=10,
            verbose=1,
            callbacks=callbacks
    )
else:
    cnn_net = load_model('./training/epoch_2.hdf5')

    cnn_history = cnn_net.fit(
        train_dataset.take(10000).batch(32).prefetch(2),
        validation_data=val_dataset.batch(32).prefetch(2),
        epochs=10,
        verbose=1,
        callbacks=callbacks
    )

Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node 'CNN/time_distributed_1/Reshape_1' defined at (most recent call last):
    File "C:\Users\Sigurd\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\Sigurd\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "E:\OneDrive - Universitetet i Agder\kornmo-master-thesis\.venv\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "E:\OneDrive - Universitetet i Agder\kornmo-master-thesis\.venv\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "E:\OneDrive - Universitetet i Agder\kornmo-master-thesis\.venv\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "E:\OneDrive - Universitetet i Agder\kornmo-master-thesis\.